# Latent Dirichlet Allocation

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import seaborn as sns 
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

import gc

In [ ]:
TRAIN_PATH = "../input/tabular-playground-series-dec-2021/train.csv"
TEST_PATH = "../input/tabular-playground-series-dec-2021/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-dec-2021/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

TARGET = "Cover_Type"
ID = "Id"
SEED = 2021
MAX_RUNTIME_SECS = 60 * 10

TREE_METHOD = 'gpu_hist'
N_COMPONENT = 2

In [ ]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

In [ ]:
lda = LDA(n_components=N_COMPONENT)
train_lda = lda.fit_transform(train.drop(columns=TARGET),train[TARGET])
test_lda = lda.transform(test)

In [ ]:
y = train[TARGET]
X = train_lda
X_test = test_lda

In [ ]:
model = XGBClassifier(tree_method=TREE_METHOD) 
model.fit(X, y)

In [ ]:
model.get_params()

In [ ]:
model.get_num_boosting_rounds()

In [ ]:
model.get_xgb_params()

In [ ]:
preds_y = model.predict(X_test)

In [ ]:
TITLE = "LDA2"
SUBMISSION_PATH = f"submission_{TITLE}.csv"

submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[TARGET] = preds_y
submission.to_csv(SUBMISSION_PATH, index=False)
submission.head()